In [5]:
#import datasets
from aif360.datasets import StandardDataset
from aif360.datasets import CompasDataset
#import fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt


pip install 'aif360[inFairness]'


In [6]:
#load COMPAS dataset

try:
    compas = CompasDataset(
        protected_attribute_names=['sex', 'race'],
        privileged_classes=[['Female'], ['Caucasian']], 
        features_to_keep=['age', 'c_charge_degree', 'race', 'age_cat', 
                          'sex', 'priors_count', 'days_b_screening_arrest', 'c_charge_desc'],
        features_to_drop=[],
        categorical_features=['age_cat', 'c_charge_degree', 'c_charge_desc'],
        label_name='two_year_recid'
    )
    print("Dataset loaded successfully!")

    #returns the dataframe and the metadata in a tuple
    df, meta = compas.convert_to_dataframe()

except Exception as e:
    print(f"Error loading COMPAS dataset: {e}")

Dataset loaded successfully!


In [7]:
# copy dataset to ensure original remains unchanged
df = df.copy()

#separate features and labels
features = ['race', 'sex', 'priors_count', 'c_charge_degree=F', 'c_charge_degree=M']
target = 'two_year_recid' #binary target where 0 means does not offend, 1 means offends

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

X_test_indices = X_test.index

#retrive each instance's group membership before scaling X_test to make predictions 
#scaling will make it lose the index information to retrieve this information
grp_membership = df.loc[X_test_indices, 'race'].values
print("\n group membership: ", grp_membership, "\n")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(solver='lbfgs')
model.fit(X_train_scaled, y_train)

#predicted class labels 0 or 1
y_pred = model.predict(X_test_scaled)


 group membership:  [1. 1. 0. ... 0. 0. 0.] 



In [8]:
#predictions array
predictions = y_pred 
print("predictions: ", predictions)

#ground truth labels array
ground_truth = y_test.values 
print("ground truth labels: ", ground_truth)

#group membership array defined above where model is trained.
print("group membership: ", grp_membership)

predictions:  [0. 0. 0. ... 0. 0. 0.]
ground truth labels:  [0. 0. 1. ... 0. 1. 1.]
group membership:  [1. 1. 0. ... 0. 0. 0.]


#### Variant of new metric:

- bi ranges 0,1,-1

In [46]:
def new_metric(arr_pred, arr_true, arr_grp):
    grp_priv = [[], []]
    grp_unpriv = [[], []]

    #j is the number of unique groups in arr_grp - an implicit parameter.
    j = len(set(arr_grp))
    #print("total number of unique groups: ", j)

    for i, label in enumerate(arr_grp):
        #for privileged class
        if label == 1.0:
            #add the corresponding prediction + gt label for that class using the index associated with that label
            grp_priv[0].append(arr_pred[i])
            grp_priv[1].append(arr_true[i])
        
        #for unprivileged class 
        else:
            grp_unpriv[0].append(arr_pred[i])
            grp_unpriv[1].append(arr_true[i])
    
    #print("Privileged group: ", grp_priv)
    #print("Unprivileged group: ", grp_unpriv)
    
    priv_indiv_bi = [] #stores individual benefit value of each instance
    priv_grp_bi = 0 #tracks total benefit for group
    
    #1. for each index in a group calculate the benefit, bi
    for pred, gt in zip(grp_priv[0], grp_priv[1]):
        #calculate benefit for each instance in a group with new range, 0,1,-1 
        indiv_benefit = (int(pred) - int(gt)) 
        
        #2. Sum the total benefit of each group
        priv_grp_bi += indiv_benefit
        
        #3. divide by size of group which is total number of instances in each group.
        priv_av_bi = priv_grp_bi / len(grp_priv[0]) #[0] has predictions which will give that number.

        #store individual benefit of each instance in a list
        priv_indiv_bi.append(indiv_benefit)
        
    #print("all bi scores for privileged instances:\n", priv_indiv_bi)
    #print(priv_grp_bi)
    #print(priv_av_bi)

    unpriv_indiv_bi = []
    unpriv_grp_bi = 0
    for pred, gt in zip(grp_unpriv[0], grp_unpriv[1]):
        indiv_benefit = (int(pred) - int(gt))  
        unpriv_grp_bi += indiv_benefit
        unpriv_av_bi = unpriv_grp_bi / len(grp_priv[0])
        unpriv_indiv_bi.append(indiv_benefit)

    #print("\nall bi scores for unprivileged instances:\n", unpriv_indiv_bi)
    #print(unpriv_grp_bi)
    #print(unpriv_av_bi)
   
    #4. division result is divided by the sum of g1 and g2 - J
    result = int(priv_av_bi + unpriv_av_bi) / j

    #print("new metric value: ", result)
    #return  priv_av_bi, unpriv_av_bi to see individual benefit for each group type
    return result

#new_metric(predictions, ground_truth, grp_membership)


#### Test cases for metric from generated arrays

In [47]:
# print("results from COMPAS data classification:\n", new_metric(predictions, ground_truth, grp_membership))

# test case 1: arr_grp1_asc, arr_true1, arr_pred1 
print("results from test case 1 (random classifier) - all arrays with 50/50:\n", new_metric(arr_grp1_asc, arr_true1, arr_pred1) )

# # test case 2: arr_grp1_desc, arr_true1_asc, arr_pred1_asc 
# print("results from test case 2 - all arrays with 50/50 but arr_grp has 1s before 0s:\n", new_metric(arr_grp1_desc, arr_true1_asc, arr_pred1_asc))

# # test case 3: arr_grp2, arr_true2, arr_pred2 
# print("results from test case 3 - arr_true is 80/20, rest are 50/50:\n", new_metric(arr_grp2, arr_true2, arr_pred2) )

# # test case 4: arr_grp3, arr_true3, arr_pred3 
# print("results from test case 4 - arr_pred is 80/20, rest are 50/50:\n", new_metric(arr_grp3, arr_true3, arr_pred3))

# #test case 5:
# print("results from test case 5:\n", new_metric(arr_grp4, arr_true4, arr_pred4) )

# #test case 6:
# print("results from test case 6:\n", new_metric(arr_grp5, arr_true5, arr_pred5) )

# #test case 7:
# print("results from test case 7:\n", new_metric(arr_grp6, arr_true6, arr_pred6) )

# #test case 8: 
# print("results from test case 8 :\n", new_metric(arr_grp7, arr_true7, arr_pred7) )

# #test case 9:
# print("results from test case 9 :\n", new_metric(arr_grp8, arr_true8, arr_pred8))

results from test case 1 (random classifier) - all arrays with 50/50:
 0.0


In [48]:
#function to generate arrays synthetically by using fixed distributions:

#function takes in the size of array and the type of distribution to generate
#also takes as input a flag to swap the default mapping of probabilities associated with the distribution
#also takes as input the order for correct 0s and 1s placement.
def gen_fixed_dist_combinations(num_of_instances, grp_dist, true_dist, pred_dist, 
                          pred_order='asc', true_order='asc', grp_order='asc', 
                          swap_group=False, swap_gt=False, swap_pred=False):
    
    #a dictionary that stores the types of expected distributions and maps to their corresp probabilties
    distribution_mapping = {"50/50": (0.5, 0.5), 
                            "80/20": (0.8, 0.2), 
                            "90/10": (0.9, 0.1) }

    #get the given parameters of probabilties for each distribution from the dict
    group_probability = distribution_mapping.get(grp_dist)
    #print("original group probs: ", group_probability)
    gt_probability = distribution_mapping.get(true_dist)
    pred_probability = distribution_mapping.get(pred_dist)
   

    #if swap parameter is True, then it means that the first element in the dict is interpreted as being the probability for 1s
    if swap_group:
        group_probability = (group_probability[1], group_probability[0])
        #print("swapped group prob: ", group_probability)
    if swap_gt:
        gt_probability = (gt_probability[1], gt_probability[0])
    if swap_pred:
        pred_probability = (pred_probability[1], pred_probability[0])

    #calculations for fixing the 0s and 1s for each array using probabilities and array size
    group_zeroes =  int(num_of_instances * group_probability[0])
    group_ones =  num_of_instances - group_zeroes

    true_zeroes = int(num_of_instances * gt_probability[0])
    true_ones = num_of_instances - true_zeroes

    pred_zeroes = int(num_of_instances * pred_probability[0])
    pred_ones = num_of_instances - pred_zeroes

    #create predictions array based on desired order of 0s and 1s.
    if pred_order == 'asc':
        arr_pred = np.array([0] * pred_zeroes + [1] * pred_ones)
    elif pred_order == 'desc':
        arr_pred = np.array([1] * pred_zeroes + [0] * pred_ones)
    else:
        raise ValueError("prediction array order must be 'asc' or 'desc'")

    #create ground truth labels array based on desired order of 0s and 1s.
    if true_order == 'asc':
        arr_true = np.array([0] * true_zeroes + [1] * true_ones)
    elif pred_order == 'desc':
        arr_true = np.array([1] * true_zeroes + [0] * true_ones)
    else:
        raise ValueError("prediction array order must be 'asc' or 'desc'")

    #create group memberships array based on desired order of 0s and 1s.
    if grp_order == 'asc':
        arr_grp = np.array([0] * group_zeroes + [1] * group_ones)
    elif grp_order == 'desc':
        arr_grp = np.array([1] * group_zeroes + [0] * group_ones)
    else:
        raise ValueError("group membership array order must be 'asc' or 'desc'")
    
    return arr_grp, arr_true, arr_pred  


In [51]:
#test case 1
arr_grp1_asc, arr_true1, arr_pred1 = gen_fixed_dist_combinations(100, 
                                                    grp_dist = "50/50", 
                                                    true_dist = "50/50", 
                                                    pred_dist = "50/50", 
                                                    pred_order = "asc", 
                                                    true_order = "asc", 
                                                    grp_order = 'asc', 
                                                    swap_group=True)
# print("predictions: ", arr_pred1)
# print("grount truth labels: ", arr_true1)
# print("group membership is 50/50: ", arr_grp1_asc)

# #test case 2
# arr_grp1_desc, arr_true1_asc, arr_pred1_asc = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "50/50", 
#                                                     true_dist = "50/50", 
#                                                     pred_dist = "50/50", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = "desc", #changed
#                                                     swap_group=True)
# #test case 3
# arr_grp2, arr_true2, arr_pred2 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "80/20", #changed
#                                                     true_dist = "50/50", 
#                                                     pred_dist = "50/50", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = "asc", 
#                                                     swap_group=True)
# #test case 4
# arr_grp3, arr_true3, arr_pred3 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "50/50", 
#                                                     true_dist = "80/20", #changed
#                                                     pred_dist = "50/50", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)

# #test case 5:
# arr_grp4, arr_true4, arr_pred4 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "50/50", 
#                                                     true_dist = "50/50",
#                                                     pred_dist = "80/20", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)

# #test case 6:
# arr_grp5, arr_true5, arr_pred5 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "80/20", 
#                                                     true_dist = "80/20",
#                                                     pred_dist = "50/50", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)
# #test case 7:
# arr_grp6, arr_true6, arr_pred6 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "80/20", 
#                                                     true_dist = "50/50",
#                                                     pred_dist = "80/20", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)

# #test case 8:
# arr_grp7, arr_true7, arr_pred7 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "50/50", 
#                                                     true_dist = "80/20",
#                                                     pred_dist = "80/20", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)

# #test case 9:
# arr_grp8, arr_true8, arr_pred8 = gen_fixed_dist_combinations(100, 
#                                                     grp_dist = "80/20", 
#                                                     true_dist = "80/20",
#                                                     pred_dist = "80/20", 
#                                                     pred_order = "asc", 
#                                                     true_order = "asc", 
#                                                     grp_order = 'asc', 
#                                                     swap_group=True)

predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
grount truth labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
group membership is 50/50:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
